In [1]:
import numpy as np
import xarray as xr
import os

In [2]:
# setting the path
data = '../Result/'
src = '../../WP1/'

In [3]:
# loading the Land-use map in the past years
LU_1904 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/1904_7PFT.nc',decode_times=False).LC.values[0]
LU_1956 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/1956_7PFT.nc',decode_times=False).LC.values[0]
LU_1982 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/1982_7PFT.nc',decode_times=False).LC.values[0]
LU_1995 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/1995_7PFT.nc',decode_times=False).LC.values[0]
LU_2000 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/2000_7PFT.nc',decode_times=False).LC.values[0]
LU_2010 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/2010_7PFT.nc',decode_times=False).LC.values[0]
LU_2015 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/7classfinal/2015_7PFT.nc',decode_times=False).LC.values[0]
LU_2019 = xr.open_dataset('/work/ychen/ycmeet/SPOT_Classification/finalmap/TWD_1997_TM_Taiwan/7_class/2019_500m_7PFT.nc',decode_times=False).LU_TYPE.values[0]

In [4]:
# Reclassify to four types
#       1      4      2      13     8     26      52
#     Forest Agri  Grass   Water  City   Soil    Agri(I)
def Reclassify(map):
    Temp_array = np.zeros((map.shape[0], map.shape[1]))
    O_type = [1, 2, 4, 8, 13, 26, 52]
    N_tpye = [1, 2, 8, 13]
    for i in range(map.shape[0]):
        for j in range(map.shape[1]):
            if map[i, j] == O_type[0]:
                Temp_array[i, j] = N_tpye[0]
            elif (map[i, j] == O_type[1] or map[i, j] == O_type[2] or map[i, j] == O_type[6]):
                Temp_array[i, j] = N_tpye[1] 
            elif (map[i, j] == O_type[3] or map[i, j] == O_type[5]):
                Temp_array[i, j] = N_tpye[2]
            elif map[i, j] == O_type[4]:
                Temp_array[i, j] = N_tpye[3]
    return Temp_array

In [8]:
## Definition of functions
# "LU_Processing" is used to show the land-use types on a map and count the number of land-use types. 
# I/O: 2-D classified map/ 1.return a input map 2.the types of classification 3.count number of land-use types.
def LU_Processing(map):
    LU = np.zeros((map.shape))
    LU[:] = map
    for i in range(LU.shape[0]):
        for j in range(LU.shape[1]):
            if LU[i,j]!=LU[i,j]:
                LU[i,j]=-999
    unique, counts = np.unique(LU, return_counts=True)
    return LU,unique, counts

# landing the block mesh of the Taiwan shape
block_mesh = np.load(src + 'block_mesh.npy')

# "LUSS" is used to calculate the posibility of future land-use change(PLC) in LU_code = [1, 2, 8, 13]
# I/O: 2-D classified map/ 4-D PLC map(base on 7 classifications)
# LU_codes are represented as following [1] Forest [2]Grass+Agriculture [8]City [13]Water 

def LUSS4T(map):
    LU = map
    array = np.zeros((4,900, 747))
    LU_code= [1, 2, 8, 13]

    for i in range(LU.shape[0]):
        for j in range(LU.shape[1]):
            for k in range(len(LU_code)):
                if LU[i,j] == LU_code[k]:
                    array[k,i,j]=array[k,i,j]+1
                    try:
                        array[k,i+1,j]=array[k,i+1,j]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i,j+1]=array[k,i,j+1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i+1,j+1]=array[k,i+1,j+1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j]=array[k,i-1,j]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i,j-1]=array[k,i,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j-1]=array[k,i-1,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i+1,j-1]=array[k,i+1,j-1]+1
                    except IndexError:
                        pass
                    try:
                        array[k,i-1,j+1]=array[k,i-1,j+1]+1
                    except IndexError:
                        pass
                    
                        
    for i in range(4):
        array[i] = array[i]*block_mesh
        
    PLUSS = (array/9*100)*block_mesh
    return PLUSS


In [ ]:
# PLC map in each year
PLC_1904 = LUSS4T(LU_1904) 
PLC_1956 = LUSS4T(LU_1956) 
PLC_1982 = LUSS4T(LU_1982) 
PLC_1995 = LUSS4T(LU_1995) 
PLC_2000 = LUSS4T(LU_2000) 
PLC_2010 = LUSS4T(LU_2010) 
PLC_2015 = LUSS4T(LU_2015) 
PLC_2019 = LUSS4T(LU_2019)

# PLC map from 1904 to 2015
TLUSS = (LUSS4T(LU_1904)*52+
        LUSS4T(LU_1956)*26+
        LUSS4T(LU_1982)*13+
        LUSS4T(LU_1995)*5+
        LUSS4T(LU_2000)*10+
        LUSS4T(LU_2010)*5+
        LUSS4T(LU_2015))/116

TLUSS_TW = TLUSS*block_mesh

np.save(data + 'TLUSS_Percentage_4T.npy', TLUSS_TW)